In [142]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

In [143]:
#Het inladen van de CSV bestand.
filename = "Case_PRB_Europe_Total_Deaths.csv"
df = pd.read_csv(filename)

#De loop_back variabel dat aangeeft hoever het LSTM-model terug moet kijken tijdens
look_back = 5

In [144]:
#convert datetime string to datetime object
#Assign index to date column
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)
df.set_axis(df["date"], inplace = True)

In [145]:
#Het ophalen van de totale doden en het reshapen van de data
death_data = df["total_deaths"].values
death_data = death_data.reshape((-1, 1))

#Aangeven wat de splitsing van de data zal zijn
#Een berekening waarbij er een int uit komt gebruikt zal worden om aan te geven vanaf welke index de data gesplits moet worden.
split_percent = 0.80
split = int(split_percent*len(death_data))

#Het splitsen van de 'total_death' data met de voorheen berekende split variabel.
death_train = death_data[:split]
death_test = death_data[split:]

#Het splitsen van de 'date' data met de voorheen berekende split variabel.
date_train = df["date"][:split]
date_test = df["date"][split:]

In [146]:
#Het kleine setjes aan data van de dataset zodat deze één voor één in een sequence door het neuraal netwerk heen gevoerd kan worden.
#bijvoorbeeld [1,2,3,4,5] -> [[1,2,3],[2,3,4],[3,4,5]]
train_generator = TimeseriesGenerator(death_train, death_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(death_test, death_test, length=look_back, batch_size=1)

In [147]:
#Het bouwen van het model met een LSTM laag.
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(look_back,1)))
model.add(Dropout(rate=0.2))#De dropout laag helpt met het verminderen van de loss.
model.add(Dense(1))#De output laag.
model.compile(optimizer = "adam", loss="mse", metrics = ["accuracy"])#Het compilen van het model.

#Het trainen van het model.
model.fit_generator(train_generator, epochs=30, verbose=1)


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8256\214584793.py:9: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/30
12/12 [==============================] - 1s 3ms/step - loss: 15061089280.0000 - accuracy: 0.1839
Epoch 2/30
12/12 [==============================] - 0s 5ms/step - loss: 13378637824.0000 - accuracy: 0.1839
Epoch 3/30
12/12 [==============================] - 0s 3ms/step - loss: 7807358976.0000 - accuracy: 0.1839
Epoch 4/30
12/12 [==============================] - 0s 4ms/step - loss: 2035812864.0000 - accuracy: 0.1839
Epoch 5/30
12/12 [==============================] - 0s 4ms/step - loss: 552113536.0000 - accuracy: 0.1839 
Epoch 6/30
12/12 [==============================] - 0s 4ms/step - loss: 894009408.0000 - accuracy: 0.1839
Epoch 7/30
12/12 [==============================] - 0s 4ms/step - loss: 553768640.0000 - accuracy: 0.1839
Epoch 8/30
12/12 [==============================] - 0s 4ms/step - loss: 536813216.0000 - accuracy: 0.1839
Epoch 9/30
12/12 [==============================] - 0s 3ms/step - loss: 496050080.0000 - accuracy: 0.1839
Epoch 10/30
12/12 [====================

In [1]:
#Het voorspellen van de waardes met het getrainde model.
prediction = model.predict(test_generator)

#Het reshapen van de data om deze in de grafiek te plotten.
death_train = death_train.reshape((-1))
death_test = death_test.reshape((-1))
prediction = prediction.reshape((-1))

#Het plotten van de resultaten in een grafiek.
trace1 = go.Scatter(
    x = date_train,
    y = death_train,
    mode = "lines",
    name = "Data"
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = "lines",
    name = "Prediction"
)
trace3 = go.Scatter(
    x = date_test,
    y = death_test,
    mode="lines",
    name = "Ground Truth"
)
layout = go.Layout(
    title = "Total deaths prediction",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Total Deaths"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

NameError: name 'model' is not defined